In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=722.00, M2=1957.00


In [12]:
only = gp.Model("only")
only.setParam('OutputFlag', 0)
only.setParam("MIPGap", 1e-7)

x_ind = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x_individual")
yp_ind = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus_individual")
ym_ind = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus_individual")

M = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")

only.update()

obj = gp.quicksum(P_DA[t] * x_ind[i, t] for i in range(I) for t in range(T)) + gp.quicksum(1 / S * (P_RT[t, s] * yp_ind[i, t, s] - P_PN[t] * ym_ind[i, t, s]) for i in range(I) for t in range(T) for s in range(S))
only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - x_ind[i, t] == yp_ind[i, t, s] - ym_ind[i, t, s])
    only.addConstr(yp_ind[i, t, s] <= R[i, t, s])
    only.addConstr(yp_ind[i, t, s] <= M * z[i, t, s])
    only.addConstr(ym_ind[i, t, s] <= M * (1 - z[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {only.objVal}")
else:
    print("No optimal solution found.")
    
x_ind_vals = np.array([[x_ind[i, t].x for t in range(T)] for i in range(I)])
y_plus_ind_vals = np.array([[[yp_ind[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
y_minus_ind_vals = np.array([[[ym_ind[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])

Optimal solution found!
Objective value: 1543726.3813601604


In [13]:
agg = gp.Model("agg")
agg.setParam('OutputFlag', 0)
agg.setParam("MIPGap", 1e-7)

a_agg = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp_agg = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm_agg = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
prob = np.array([1 / S for s in range(S)])

agg.update()

obj = gp.quicksum(P_DA[t] * a_agg[t] for t in range(T)) + gp.quicksum(prob[s] * (P_RT[t, s] * bp_agg[t, s] - P_PN[t] * bm_agg[t, s]) for t in range(T) for s in range(S))
agg.setObjective(obj, GRB.MAXIMIZE)

for t, s in product(range(T), range(S)):
    agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a_agg[t] == bp_agg[t, s] - bm_agg[t, s])
    agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= bp_agg[t, s])
    agg.addConstr(bp_agg[t, s] <= M * z[t, s])
    agg.addConstr(bm_agg[t, s] <= M * (1 - z[t, s]))

agg.optimize()

if agg.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {agg.objVal}")
else:
    print("No optimal solution found.")
    
a_agg_vals = np.array([a_agg[t].x for t in range(T)])
b_plus_ind_vals = np.array([[bp_agg[t, s].x for s in range(S)] for t in range(T)])
b_minus_ind_vals = np.array([[bm_agg[t, s].x for s in range(S)] for t in range(T)])

Optimal solution found!
Objective value: 1567561.8840047559


In [16]:
set = gp.Model("Settlement") 
set.setParam('OutputFlag', 0)
set.setParam("PoolSolutions", 15)
set.setParam("PoolSearchMode", 2)
set.setParam("PoolGap", 0.05)

a = set.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
x = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
zy = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
ep = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
em = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
ze = set.addVars(T, S, vtype=GRB.BINARY, name="z_e") 
set.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(1/S * (P_RT[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))

set.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(R[i, t, s] - x[i, t, s] == yp[i, t, s] - ym[i, t, s])
    set.addConstr(yp[i, t, s] <= R[i, t, s])
    set.addConstr(ep[i,t,s] == yp[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i))
    set.addConstr(em[i,t,s] == ym[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i))
    set.addConstr(gp.quicksum(d[i, j, t, s] for j in range(I) if j != i) <= yp[i, t, s])
    set.addConstr(gp.quicksum(d[j, i, t, s] for j in range(I) if j != i) <= ym[i, t, s])
    set.addConstr(d[i, i, t, s] == 0)
    set.addConstr(yp[i, t, s] <= M1 * zy[i, t, s])
    set.addConstr(ym[i, t, s] <= M1 * (1 - zy[i, t, s]))
    
for t, s in product(range(T), range(S)):
    set.addConstr(a[t] == gp.quicksum(x[i, t, s] for i in range(I)))
    set.addConstr(bp[t,s] == gp.quicksum(ep[i, t, s] for i in range(I)))
    set.addConstr(bm[t,s] == gp.quicksum(em[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t, s] for i in range(I)) == gp.quicksum(ep[i, t, s] for i in range(I)) - gp.quicksum(em[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= M2 * ze[t, s])
    set.addConstr(gp.quicksum(em[i, t, s] for i in range(I)) <= M2 * (1 - ze[t, s]))

set.optimize()

if set.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {set.objVal}")
else:
    print("No optimal solution found.")

Optimal solution found!
Objective value: 1567561.8840047559


In [ ]:
#